In [3]:
import requests
from bs4 import BeautifulSoup
from pprint import pprint
import json
import sqlite3
import gzip
from io import BytesIO
import logging

# Configurar logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

In [4]:
# Célula 2 - Configurações
CONFIG = {
    'batch_size': 1000,  # Registros por commit
    'db_name': 'goodreads.db',
    'base_url': 'https://mengtingwan.github.io/data/goodreads.html'
}

In [5]:

# Célula 3 - Função de criação do banco
def criar_banco():
    """
    Cria o banco de dados SQLite e a estrutura da tabela books
    
    Returns:
        tuple: (conexão com o banco, cursor)
    """
    banco = sqlite3.connect(CONFIG['db_name'])
    cursor = banco.cursor()
    
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS books (
        -- Identificadores
        book_id TEXT PRIMARY KEY,
        work_id TEXT,
        
        -- Informações básicas
        title TEXT,
        title_without_series TEXT,
        description TEXT,
        
        -- ISBNs e identificadores
        isbn TEXT,
        isbn13 TEXT,
        asin TEXT,
        kindle_asin TEXT,
        
        -- Metadados do livro
        authors TEXT,
        publisher TEXT,
        num_pages INTEGER,
        format TEXT,
        
        -- Datas de publicação
        publication_year INTEGER,
        publication_month INTEGER,
        publication_day INTEGER,
        
        -- Avaliações
        average_rating REAL,
        ratings_count INTEGER,
        text_reviews_count INTEGER,
        
        -- Informações adicionais
        language_code TEXT,
        country_code TEXT,
        is_ebook TEXT,
        series TEXT,
        
        -- URLs
        image_url TEXT,
        link TEXT
    )
    ''')
    
    banco.commit()
    return banco, cursor

In [6]:
# Célula 4 - Função de processamento de dados
def processo_pegar_dados(url):
    """
    Processa um arquivo GZ e insere no banco de dados
    
    Args:
        url (str): URL do arquivo .gz para processar
    """
    arquivo = url.split('/')[-1]
    try:
        logging.info(f"Processo Iniciado do arquivo {arquivo}")
        response = requests.get(url, stream=True)
        response.raise_for_status()

        banco, cursor = criar_banco()

        stream = BytesIO(response.raw.read())

        logging.info("Processando dados...")
        with gzip.GzipFile(fileobj=stream) as gz_file:
            count = 0
            for line in gz_file:
                try:
                    book = json.loads(line)
                    
                    book_data = (
                        book.get('book_id'),
                        book.get('work_id'),
                        book.get('title'),
                        book.get('title_without_series'),
                        book.get('description'),
                        book.get('isbn'),
                        book.get('isbn13'),
                        book.get('asin'),
                        book.get('kindle_asin'),
                        json.dumps(book.get('authors', [])),
                        book.get('publisher'),
                        book.get('num_pages'),
                        book.get('format'),
                        book.get('publication_year'),
                        book.get('publication_month'),
                        book.get('publication_day'),
                        book.get('average_rating'),
                        book.get('ratings_count'),
                        book.get('text_reviews_count'),
                        book.get('language_code'),
                        book.get('country_code'),
                        book.get('is_ebook'),
                        json.dumps(book.get('series', [])),
                        book.get('image_url'),
                        book.get('link')
                    )
                    
                    cursor.execute('''
                    INSERT OR REPLACE INTO books (
                        book_id, work_id, ... , link
                    )
                    VALUES (?, ?, ?, ?, ?, ...)
                    ''', book_data)

                    count += 1
                    if count % CONFIG['batch_size'] == 0:
                        banco.commit()
                        logging.info(f"Processados {count} livros.")
                    
                except json.JSONDecodeError as e:
                    logging.error(f"Erro ao decodificar Json: {e}")
                    continue
    
    except Exception as e:
        logging.error(f"Error: {e}")
    finally:
        if 'banco' in locals():
            banco.close()

In [7]:
# Célula 5 - Função de coleta de links
def link_livros_generos():
    """
    Coleta todos os links de arquivos .gz da página do Goodreads
    
    Returns:
        dict: Dicionário com links separados por categoria
    """
    links_genero = {
        'livros': [],
        'interacoes': [],
        'reviews': []
    }

    try:
        response = requests.get(CONFIG['base_url'])
        response.raise_for_status()

        soup = BeautifulSoup(response.text, 'html.parser')
        encontrar_genero = soup.find('h3', text=lambda t: t and 'By Genre' in t)

        if encontrar_genero:
            todos_links = soup.find_all('a', href=lambda href: href and href.endswith('.gz'))
            
            for link in todos_links:
                href = link.get('href')
                if 'books' in href:
                    links_genero['livros'].append(href)
                elif 'interactions' in href:
                    links_genero['interacoes'].append(href)
                elif 'reviews' in href:
                    links_genero['reviews'].append(href)

            return links_genero

    except Exception as e:
        print(f"Erro: {e}")
        return None

In [8]:
# Célula 6 - Função de estatísticas
def mostrar_estatisticas():
    """
    Mostra estatísticas dos dados no banco
    """
    banco, cursor = criar_banco()
    try:
        cursor.execute("SELECT COUNT(*) from books")
        total = cursor.fetchone()[0]
        
        cursor.execute("SELECT COUNT(*) from books WHERE is_ebook = 'true'")
        ebooks = cursor.fetchone()[0]
        
        logging.info(f"Total de livros: {total}")
        logging.info(f"Total de ebooks: {ebooks}")
    finally:
        banco.close()

In [10]:
# Célula 7 - Execução principal
# Você pode executar células individuais para testar
# ou executar todo o processo

# Coletar links
links = link_livros_generos()
if links:
    print("Links coletados:")
    pprint(links)

# Processar um arquivo específico
url_teste = links['livros'][1]  # primeiro arquivo de livros
processo_pegar_dados(url_teste)

# Verificar estatísticas
mostrar_estatisticas()

C:\Users\Felipe\AppData\Local\Temp\ipykernel_20132\2585862438.py:20: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  encontrar_genero = soup.find('h3', text=lambda t: t and 'By Genre' in t)
2025-01-16 10:28:41,728 - INFO - Processo Iniciado do arquivo goodreads_books_children.json.gz


Links coletados:
{'interacoes': ['https://datarepo.eng.ucsd.edu/mcauley_group/gdrive/goodreads/goodreads_interactions_dedup.json.gz',
                'https://datarepo.eng.ucsd.edu/mcauley_group/gdrive/goodreads/byGenre/goodreads_interactions_children.json.gz',
                'https://datarepo.eng.ucsd.edu/mcauley_group/gdrive/goodreads/byGenre/goodreads_interactions_comics_graphic.json.gz',
                'https://datarepo.eng.ucsd.edu/mcauley_group/gdrive/goodreads/byGenre/goodreads_interactions_fantasy_paranormal.json.gz',
                'https://datarepo.eng.ucsd.edu/mcauley_group/gdrive/goodreads/byGenre/goodreads_interactions_history_biography.json.gz',
                'https://datarepo.eng.ucsd.edu/mcauley_group/gdrive/goodreads/byGenre/goodreads_interactions_mystery_thriller_crime.json.gz',
                'https://datarepo.eng.ucsd.edu/mcauley_group/gdrive/goodreads/byGenre/goodreads_interactions_poetry.json.gz',
                'https://datarepo.eng.ucsd.edu/mcauley_group/

2025-01-16 10:28:56,624 - INFO - Processando dados...
2025-01-16 10:28:56,625 - ERROR - Error: near ".": syntax error
2025-01-16 10:28:56,639 - INFO - Total de livros: 0
2025-01-16 10:28:56,639 - INFO - Total de ebooks: 0
